# 1. Data collection

## 1.1 Get the list of animes

In [2]:
def getUrls(start, stop): 

    urls = []

    for i in range(start, stop):
        url = 'https://myanimelist.net/topanime.php?limit='+str(i*50)
        r = requests.get(url)
        html_content = r.text
        soup = bs(html_content, 'lxml')
        links = soup.find_all('h3') 

        for anime in links[:-3]:
            if anime.find('a'):
                urls.append(anime.find('a')['href'])
    return urls

In [3]:
f = open("topAnime.txt", 'w', encoding="utf8")
f.write('\n'.join(getUrls(0,400)))
f.close

<function TextIOWrapper.close()>

## 1.2 Crawl animes

In [2]:
def saveHtml(page):
#saving the HTML file of page 'page' in the corresponding folder

    subfolder = "downloaded_Html/page_{}".format(page)
    os.makedirs(subfolder)

    f = open("topAnime.txt", 'r', encoding="utf8")
    lines = f.readlines()[(page-1)*50:(page)*50]
    lines = [line.rstrip() for line in lines]
    f.close

    i = 1+50*(page-1)
    for link in lines:
        html = requests.get(link)
        
        if html.status_code != 200:
            while(html.status_code != 200): 
                html = requests.get(link)
        
        file_name = '{}/{}.html'.format(subfolder, i)
        g = open(file_name, 'w', encoding="utf8")
        g.write(html.text)
        g.close
        i += 1

In [3]:
for i in range(379,400):
    saveHtml(i)

## 1.3 Parse downloaded pages

In [2]:
def getTitle(anime):
    return anime.strong.contents[0]

In [3]:
def getType(anime):
    return anime.find(text = 'Type:').find_next('a').contents[0]

In [4]:
def getNumEpis(anime):
    if anime.find(text = 'Episodes:').next_element.strip() != "Unknown":
        return int(anime.find(text = 'Episodes:').next_element.strip())
    else :
        return []

In [5]:
def getStart(anime):
    date = anime.find(text = 'Aired:').next_element.strip()
    
    if date != "Not available":
        if len(date.split(" to ")[0]) > 8:
            return dt.strptime(date.split(" to ")[0], '%b %d, %Y' )
        elif len(date.split(" to ")[0]) == 4:
            return dt.strptime(date.split(" to ")[0], '%Y' )
        elif len(date.split(" to ")[0]) == 8:
            return dt.strptime(date.split(" to ")[0], '%b %Y' )
        else:
            return pd.to_datetime(np.NaN, errors='coerce')
    else :
        return []

In [6]:
def getEnd(anime):
    date = anime.find(text = 'Aired:').next_element.strip()
    
    if date != "Not available":
        if len(date)>12 and len(date.split(" to ")[1]) > 8 and date.split(" to ")[1] != "?":
            return dt.strptime(date.split(" to ")[1], '%b %d, %Y' )
        elif len(date)>12 and len(date.split(" to ")[1]) == 4 and date.split(" to ")[1] != "?":
            return dt.strptime(date.split(" to ")[1], '%Y' )
        elif len(date)>12 and len(date.split(" to ")[1]) == 8 and date.split(" to ")[1] != "?":
            return dt.strptime(date.split(" to ")[1], '%b %Y' )
        else:
            return pd.to_datetime(np.NaN, errors='coerce')
    else :
        return []        

In [7]:
def getNumMemb(anime):
    animeNumMembers = anime.find(text = 'Members:').next_element
    return int(animeNumMembers.replace('n', '').replace(',', '').strip())

In [8]:
def getScore(anime):
    if anime.find(text = 'Score:').find_next('span').contents[0] != 'N/A':
        animeScore = anime.find(text = 'Score:').find_next('span').contents
        return float(animeScore[0])
    else :
        return []            

In [9]:
def getUsers(anime):
    animeUsers = anime.find(text = 'Score:').find_next('span').find_next('span').contents
    if animeUsers[0] != 'Ranked:':
        return int(animeUsers[0])
    else :
        return []      

In [10]:
def getRank(anime):
    animeRank = anime.find(text = 'Ranked:').next_element
    if animeRank.replace('\n', '').strip() != 'N/A':
        return int(animeRank.replace('\n', '').replace('#', '').strip())
    else :
        return [] 

In [11]:
def getPopularity(anime):
    animePopularity = anime.find(text='Popularity:').next_element
    return int(animePopularity.replace("\n","").replace('#', '').strip())

In [12]:
def getDescription(anime):
    animeDescription = anime.find(text = 'Synopsis').find_next('p').text
    return animeDescription.replace("\n","")

In [13]:
def getRelated(anime):
    table = anime.find(text = 'Related Anime')
    animeRelated = []

    if(table != None):    
        table = table.find_next('table')
        table = table.find_all('a')

        for el in table:
            animeRelated.append(el.text)
                
    return animeRelated

In [14]:
def getCharact(anime):
    table = anime.find(text = 'Characters & Voice Actors').find_next('div')
    table = table.find_all('table')

    animeChar = []

    for el in table:
        people = el.find_all('h3')
        for person in people:
            animeChar.append(person.text)
        
    return animeChar

In [15]:
def getVoices(anime):
    table = anime.find(text = 'Characters & Voice Actors').find_next('div')
    table = table.find_all('h3')

    animeVoices = []

    for el in table:
        people = el.find_next('table')
        for person in people:
            animeVoices.append(person.find('a').text)
        
    return animeVoices

In [16]:
def getStaff(anime):
    
    table = anime.find_all(text = 'Staff')[1].find_next("div", {"class": "detail-characters-list clearfix"})
    animeStaff = []
    if(table != None):    
        table = table.find_all("table")
        for el in table:
            x = el.find_all("td")[1]
            person = [x.find("a").text, x.find("small").text]
            animeStaff.append(person)
    
    return animeStaff

In [17]:
pages = sorted(os.listdir('downloaded_Html')[1:], key = lambda page : int(page.split("_")[1]))

title = []
typ = []
numEpisode = []
start = []
end = []
numMembers = []
score = []
users = []
rank = []
popularity = []
synopsis = []
related = []
char = []
voices = []
staff = []

In [18]:
for page in pages:
    htmls = os.listdir('downloaded_Html/{}'.format(page))
    for i in range(1,1+len(htmls)):
        f = open("downloaded_Html/{}/{}.html".format(page,50*(int(page.split("_")[1])-1)+i), 'r', encoding="utf8")
        anime = bs(f, 'lxml')
        title.append(getTitle(anime))
        typ.append(getType(anime))
        numEpisode.append(getNumEpis(anime))
        start.append(getStart(anime))
        end.append(getEnd(anime))
        numMembers.append(getNumMemb(anime))
        score.append(getScore(anime))
        users.append(getUsers(anime))
        rank.append(getRank(anime))
        popularity.append(getPopularity(anime))
        synopsis.append(getDescription(anime))
        related.append(getRelated(anime))
        char.append(getCharact(anime))
        voices.append(getVoices(anime))
        staff.append(getStaff(anime))

In [21]:
col = ['animeTitle', 'animeType', 'animeNumEpisode','releaseDate', 'endDate', 'animeNumMembers', 'animeScore', 'animeUsers', 
       'animeRank', 'animePopularity', 'animeDescription', 'animeRelated', 'animeCharacters', 'animeVoices', 'animeStaff']

types = {'animeTitle' : 'object', 
         'animeType' : 'object', 
         'animeNumEpisode' : 'int64',
         'releaseDate' : 'datetime64', 
         'endDate' : 'datetime64', 
         'animeNumMembers' : 'int64', 
         'animeScore' : 'float64',
         'animeUsers' : 'int64', 
         'animeRank' : 'int64',
         'animePopularity' : 'int64',
         'animeDescription' : 'object',
         'animeRelated' : 'object',
         'animeCharacters' : 'object',
         'animeVoices' : 'object',
         'animeStaff' : 'object'}

df = pd.DataFrame(columns = col).astype(dtype = types) 

df.animeTitle = title 
df.animeType = typ 
df.animeNumEpisode = numEpisode 
df.releaseDate = start 
df.endDate = end 
df.animeNumMembers = numMembers 
df.animeScore = score 
df.animeUsers = users 
df.animeRank = rank 
df.animePopularity = popularity 
df.animeDescription = synopsis 
df.animeRelated = related 
df.animeCharacters = char 
df.animeVoices = voices 
df.animeStaff = staff 

In [22]:
df = df.to_csv("anime.tsv", sep='\t', index=False)

In [ ]:
def dataToTSV(page, df):
    
    subfolder = "tsv_files/page_{}".format(page)
    os.makedirs(subfolder)
    
    for i in range(len(df)):
        f = open("tsv_files/page_{}/anime_{}.tsv".format(page, i+50*(page-1)), 'w', encoding="utf8")
        tsv = csv.writer(f, delimiter = '\t')
        tsv.writerow([x for x in df.columns])
        tsv.writerow(x for x in df.iloc[i]) 

# 2. Search engine

## 2.1 Conjunctive query

In [8]:
df = pd.read_csv("anime.tsv", sep='\t')
df.head()

,animeTitle,animeType,animeNumEpisode,releaseDate,endDate,animeNumMembers,animeScore,animeUsers,animeRank,animePopularity,animeDescription,animeRelated,animeCharacters,animeVoices,animeStaff
0,Fullmetal Alchemist: Brotherhood,TV,64,2009-04-05 00:00:00,2010-07-04 00:00:00,2677887,9.16,1629051,1,3,After a horrific alchemy experiment goes wrong...,"['Fullmetal Alchemist', 'Fullmetal Alchemist',...","['Elric, Edward', 'Elric, Alphonse', 'Mustang,...","['Park, Romi', 'Kugimiya, Rie', 'Miki, Shinich...","[['Cook, Justin', 'Producer'], ['Yonai, Norito..."
1,Gintama°,TV,51,2015-04-08 00:00:00,2016-03-30 00:00:00,484270,9.09,169476,2,337,"Gintoki, Shinpachi, and Kagura return as the f...","['Gintama', 'Gintama Movie 2: Kanketsu-hen - Y...","['Sakata, Gintoki', 'Kagura', 'Shimura, Shinpa...","['Sugita, Tomokazu', 'Kugimiya, Rie', 'Sakaguc...","[['Fujita, Youichi', 'Director, Storyboard, Pl..."
2,Shingeki no Kyojin Season 3 Part 2,TV,10,2019-04-29 00:00:00,2019-07-01 00:00:00,1598256,9.09,1087519,3,33,Seeking to restore humanity's diminishing hope...,"['Shingeki no Kyojin', 'Shingeki no Kyojin Sea...","['Levi', 'Yeager, Eren', 'Ackerman, Mikasa', '...","['Kamiya, Hiroshi', 'Kaji, Yuki', 'Ishikawa, Y...","[['Yabuta, Shuuhei', 'Producer'], ['Wada, Jouj..."
3,Steins;Gate,TV,24,2011-04-06 00:00:00,2011-09-14 00:00:00,2092380,9.09,1115839,4,11,The self-proclaimed mad scientist Rintarou Oka...,"['Steins;Gate', 'ChäoS;HEAd', 'Robotics;Notes'...","['Okabe, Rintarou', 'Makise, Kurisu', 'Shiina,...","['Miyano, Mamoru', 'Imai, Asami', 'Hanazawa, K...","[['Iwasa, Gaku', 'Producer'], ['Yasuda, Takesh..."
4,Fruits Basket: The Final,TV,13,2021-04-06 00:00:00,2021-06-29 00:00:00,275939,9.07,113310,5,653,"Hundreds of years ago, the Chinese Zodiac spir...","['Fruits Basket', 'Fruits Basket 2nd Season', ...","['Souma, Kyou', 'Honda, Tooru', 'Souma, Yuki',...","['Uchida, Yuuma', 'Iwami, Manaka', 'Shimazaki,...","[['Ibata, Yoshihide', 'Director'], ['Aketagawa..."


In [9]:
synopsis = df.animeDescription

In [10]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english')) 

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [60]:
total = []
tokenizer = RegexpTokenizer(r"\w+")
ps = PorterStemmer()

for row in synopsis:
    tokens = tokenizer.tokenize(row.lower())
    for word in tokens:
        if word not in stop_words:
            total.append(ps.stem(word))
        
total = list(dict.fromkeys(total))

In [14]:
tokenizer = RegexpTokenizer(r"\w+")
ps = PorterStemmer()

def row_token(row):
    out = list()
    tokens = tokenizer.tokenize(row.lower())
    for word in tokens:
        if word not in stop_words:
            out.append(ps.stem(word))
    out = list(dict.fromkeys(out))
    return out



In [ ]:
for row in synopsis:
    for word in row_token(row):
        if word not in stop_words:
            total.append(ps.stem(word))

In [21]:
voc = {}
id = 0
for tok in total:
    voc[tok] = id
    id+=1

with open('vocabulary.csv', 'w', newline = '',encoding = 'utf8') as f:
    fieldnames = ['Word', 'term_id']
    w = csv.DictWriter(f, fieldnames = fieldnames,)
    w.writeheader()
    for key in voc:
        w.writerow({"Word": key, 'term_id': voc[key]})

In [24]:
vocabulary = pd.read_csv("vocabulary.csv")
vocabulary.Word[100]

'rewrit'

In [12]:
def setDict(d,l):
    for i in l:
        d[i] = []
    return d

def listToString(l):
    return ' '.join([str(elem) for elem in l])

In [13]:
########### OLD
inverted_index = {}
inverted_index = setDict(inverted_index, vocabulary.Word)

for i in range(3):#len(synopsis)):
    s = listToString(row_token(synopsis[i]))
    for key in vocabulary.Word:
        if s.find(str(key)) != -1:
            inverted_index[key].append(i)

NameError: name 'row_token' is not defined

In [16]:
inverted_index = {}
inverted_index = setDict(inverted_index, vocabulary.term_id)

for i in range(len(synopsis)):
    if i%100 == 0:
        print(i)
    s = listToString(row_token(synopsis[i]))
    for j in range(len(vocabulary.Word)):
        if s.find(str(vocabulary.Word[j])) != -1:
            inverted_index[vocabulary.term_id[j]].append(i)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [17]:
with open('inverted_index.csv', 'w', newline = '',encoding = 'utf8') as f:
    fieldnames = ['term_id', 'document']
    w = csv.DictWriter(f, fieldnames = fieldnames,)
    w.writeheader()
    for key in inverted_index:
        w.writerow({"term_id": key, 'document': inverted_index[key]})

In [22]:
index = pd.read_csv("inverted_index.csv")
index[100:130]

,term_id,document
100,100,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14..."
101,101,"[1, 5, 8, 10, 14, 15, 24, 94, 101, 116, 136, 1..."
102,102,"[1, 5, 14, 15, 24, 94, 101, 116, 243, 3049]"
103,103,"[1, 5, 10, 14, 15, 24, 94, 101, 1138, 1229, 12..."
104,104,"[1, 2, 4, 5, 11, 26, 29, 39, 47, 55, 57, 73, 8..."
105,105,"[1, 3, 12, 49, 70, 88, 89, 93, 145, 150, 160, ..."
106,106,"[1, 8, 14, 16, 30, 40, 49, 57, 66, 69, 74, 80,..."
107,107,"[1, 130, 292, 295, 348, 412, 434, 501, 503, 59..."
108,108,"[1, 3, 4, 5, 10, 18, 32, 53, 56, 72, 73, 79, 9..."
109,109,"[1, 5, 8, 10, 14, 15, 24, 94, 101, 116, 136, 1..."


In [ ]:
inverted_index

In [33]:
def searchWord(i):
    vocabulary = pd.read_csv("vocabulary.csv")
    return i in list(vocabulary.Word)

In [134]:
def searchEngine(query):
    vocabulary = pd.read_csv("vocabulary.csv")
    index = pd.read_csv("inverted_index.csv")
    stemQuery = row_token(query)
    
    if all([searchWord(i) for i in stemQuery]):
        ind = [np.where(vocabulary.Word == x)[0][0] for x in stemQuery]
        aux = [set(ast.literal_eval(index.document[vocabulary.term_id[i]])) for i in ind]
        return set.intersection(*aux)

In [165]:
vocabulary = pd.read_csv("vocabulary.csv")
index = pd.read_csv("inverted_index.csv")
stemQuery = row_token('back')
print(stemQuery)


if all([searchWord(word) for word in stemQuery]):
    ind = [np.where(vocabulary.Word == x)[0][0] for x in stemQuery]
    aux = [set(ast.literal_eval(index.document[vocabulary.term_id[i]])) for i in ind]
    x = set.intersection(*aux)
x
set.intersection(*aux)

['back']


{0, 2}

In [40]:
index = pd.read_csv("inverted_index.csv")
index

,term_id,document
0,horrif,"[0, 82, 99, 151, 318, 504, 506, 529, 547, 663,..."
1,alchemi,"[0, 166, 394, 455, 526, 1318, 1488, 1579, 1856..."
2,experi,"[0, 13, 23, 39, 61, 65, 70, 72, 74, 91, 93, 96..."
3,goe,"[0, 27, 55, 91, 167, 224, 299, 301, 339, 355, ..."
4,wrong,"[0, 12, 22, 37, 127, 204, 223, 331, 352, 409, ..."
...,...,...
36563,nippl,[19106]
36564,kawabiraki,[19113]
36565,boatman,[19113]
36566,kisha,[19121]


In [189]:
def documentFinder(query):
    anime = pd.read_csv("anime.tsv", sep='\t', usecols = ["animeTitle","animeDescription"])
    f = open("topAnime.txt", 'r', encoding="utf8")
    topAnimeUrls = f.readlines()
    col = ['animeTitle', "animeDescription", "Url"]
    df = pd.DataFrame(columns = col)
    title = []
    description = []
    urls = []
    
    ind = searchEngine(query)
    for i in ind:
        title.append(anime.animeTitle[i])
        description.append(anime.animeDescription[i])
        urls.append(topAnimeUrls[i])
        
    df.animeTitle = title 
    df.animeDescription = description
    df.Url = urls
    
    f.close
    return df 


In [191]:
def documentFinder(query):
    anime = pd.read_csv("anime.tsv", sep='\t', usecols = ["animeTitle","animeDescription"])
    f = open("topAnime.txt", 'r', encoding="utf8")
    topAnimeUrls = f.readlines()
    col = ['animeTitle', "animeDescription", "Url"]
    df = pd.DataFrame(columns = col)
    ind = searchEngine(query)
        
    df.animeTitle = [anime.animeTitle[i] for i in ind]
    df.animeDescription = [anime.animeDescription[i] for i in ind]
    df.Url = [topAnimeUrls[i] for i in ind]
    
    f.close
    return df 


In [192]:
documentFinder('back')

,animeTitle,animeDescription,Url
0,Fullmetal Alchemist: Brotherhood,After a horrific alchemy experiment goes wrong...,https://myanimelist.net/anime/5114/Fullmetal_A...
1,Shingeki no Kyojin Season 3 Part 2,Seeking to restore humanity's diminishing hope...,https://myanimelist.net/anime/38524/Shingeki_n...


In [1]:
import numpy as np
import pandas as pd 
import requests
import time
import csv
import os
import nltk
import ast
from nltk.corpus import stopwords
from bs4 import BeautifulSoup as bs
from datetime import datetime as dt
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer

In [2]:
import winsound


In [3]:
def saveReviews(page):
#saving the HTML file of page 'page' in the corresponding folder

    header = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36" ,
    'referer':'https://www.google.com/'
    }
    subfolder = "downloaded_Reviews/page_{}".format(page)
    os.makedirs(subfolder)

    f = open("data/topAnime.txt", 'r', encoding="utf8")
    lines = f.readlines()[(page-1)*50:(page)*50]
    lines = [line.rstrip() for line in lines]
    f.close

    i = 1+50*(page-1)
    for link in lines:
        if i != 8682:
            html = requests.get(link+"/reviews",headers=header)
            #time.sleep(2)
            if html.status_code != 200:
                winsound.Beep(400,600)
                while(html.status_code != 200): 
                    html = requests.get(link)
        
        file_name = '{}/{}.html'.format(subfolder, i)
        g = open(file_name, 'w', encoding="utf8")
        g.write(html.text)
        g.close
        i += 1

In [4]:
saveReviews(174)

In [5]:
for i in range(349,400):
    saveReviews(i)

In [6]:
import winsound
duration = 1000
freq = 400
winsound.Beep(freq,duration)